# performing "research" for the chatbot 
##### (RAG + safe Pandas function)

### Check to see if <b>jupyter notebook is running</b> and import the libaries needed

In [1]:
print("Hello World!")

Hello World!


In [2]:
# import necessary libaries
import os
import pandas as pd
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List
from langchain.embeddings import HuggingFaceBgeEmbeddings
from dotenv import load_dotenv
from pinecone import Pinecone
from pinecone import ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain_openai import ChatOpenAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
import json

c:\Users\User\anaconda3\envs\chatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### move the working directory up to access data folder

In [3]:
# checking working directory, moved it up by one so that the data folder is accessiable
%pwd

'c:\\Users\\User\\Downloads\\Personal Projects\\Generative AI Chatbot\\Building-a-Generative-AI-Chatbot-with-RAG\\research'

In [4]:
# moves working directory up
os.chdir("../")
%pwd

'c:\\Users\\User\\Downloads\\Personal Projects\\Generative AI Chatbot\\Building-a-Generative-AI-Chatbot-with-RAG'

### data understanding

In [5]:
# loading the data 
df = pd.read_csv('data/ER Wait Time Dataset.csv')
df["Visit Date"] = pd.to_datetime(df["Visit Date"])

# check column names so that data can be rewritten and stored as document
df.columns

Index(['Visit ID', 'Patient ID', 'Hospital ID', 'Hospital Name', 'Region',
       'Visit Date', 'Day of Week', 'Season', 'Time of Day', 'Urgency Level',
       'Nurse-to-Patient Ratio', 'Specialist Availability',
       'Facility Size (Beds)', 'Time to Registration (min)',
       'Time to Triage (min)', 'Time to Medical Professional (min)',
       'Total Wait Time (min)', 'Patient Outcome', 'Patient Satisfaction'],
      dtype='object')

### load <b> all data </b>from data folder (assuming consistent format)

In [6]:
# this function loads multiple files from file_path and returns it as a whole 
def load_csv_files(file_path):
    documents = []

    for file in os.listdir(file_path):
        if file.endswith(".csv"):
            file_path = os.path.join(file_path, file)

            df = pd.read_csv(file_path)
            df["Visit Date"] = pd.to_datetime(df["Visit Date"], errors="coerce")

            # the conversion of data here is hard coded, assuming datasets in the folder have the same columns/format
            for _, row in df.iterrows():
                content = f"""
                On {row['Visit Date']} at {row['Hospital Name']} (Region: {row['Region']}),
                urgency level was {row['Urgency Level']}.
                Total wait time was {row['Total Wait Time (min)']} minutes.
                Patient outcome was {row['Patient Outcome']}.
                Patient satisfaction was {row['Patient Satisfaction']}.
                """

                documents.append(
                    Document(
                        page_content=content.strip(),
                        metadata={
                            "source_file": file,
                            "hospital": row["Hospital Name"],
                            "region": row["Region"]
                        }
                    )
                )

    return documents

In [7]:
# check length of data
extracted_data = load_csv_files("data")
len(extracted_data)

5000

### <b>vector storing process</b> (in Pinecone)

In [8]:
# simplify the data and standarize for vector storing

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of document objects, return a new list of document objects
    containing only 'source' in metadata and the original page_content.
    """
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source_file")

        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )

    return minimal_docs

minimal_docs = filter_to_minimal_docs(extracted_data)
minimal_docs

[Document(metadata={'source': 'ER Wait Time Dataset.csv'}, page_content='On 2024-02-10 20:20:56 at Springfield General Hospital (Region: Urban),\n                urgency level was Medium.\n                Total wait time was 105 minutes.\n                Patient outcome was Discharged.\n                Patient satisfaction was 1.'),
 Document(metadata={'source': 'ER Wait Time Dataset.csv'}, page_content='On 2024-11-28 02:07:47 at Northside Community Hospital (Region: Rural),\n                urgency level was Medium.\n                Total wait time was 69 minutes.\n                Patient outcome was Discharged.\n                Patient satisfaction was 3.'),
 Document(metadata={'source': 'ER Wait Time Dataset.csv'}, page_content='On 2024-09-30 04:02:28 at Northside Community Hospital (Region: Rural),\n                urgency level was Low.\n                Total wait time was 203 minutes.\n                Patient outcome was Discharged.\n                Patient satisfaction was 1.'),

In [9]:
# split the documents into smaller chunks
def text_split(minimal_docs):
    text_spliter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20
    )
    text_chunks = text_spliter.split_documents(minimal_docs)
    return text_chunks

text_chunks = text_split(minimal_docs)
print(f"Number of Chunks: {len(text_chunks)}")

Number of Chunks: 5000


In [10]:
def download_embeddings():
    """
    Download and return the HuggingFace Embeddings model.
    """
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceBgeEmbeddings(
        model_name=model_name
    )
    return embeddings

embedding = download_embeddings()

C:\Users\User\AppData\Local\Temp\ipykernel_17936\1077763997.py:6: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(


In [11]:
embedding

HuggingFaceBgeEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_instruction='Represent this question for searching relevant passages: ', embed_instruction='', show_progress=False)

In [12]:
# load in the needed API keys
load_dotenv()

True

In [13]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [14]:
# assign pinecone api key to variable pc
pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

In [15]:
# creating index into pinecone if not already present
index_name = "genai-chatbot"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension = 384, # dimension of embeddings
        metric = "cosine", # cosine similarity
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

index = pc.Index(index_name)

if embeddings are already stored, <b>do not need to run the markdown code below</b>. <br>
If not duplicated embeddings will be created

docsearch = PineconeVectorStore.from_documents(
    documents = text_chunks,
    embedding = embedding,
    index_name = index_name
)

In [16]:
# load existing index

# embed each chunk and upsert the embeddings into your pinecone index
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

adding more data into existing Pinecone Index, if needed code is:

dswith = Document(
    page_content="insert new content"
    metdadata={"source": "source"}
)

docsearch.add_documents(documents=[dwsith])

### Retriving Text Chunks

In [17]:
# retrive the 3 most relevant chunks
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [18]:
# testing to see docs retrived
retrieved_docs = retriever.invoke("what is the satisfaction score of a patient visting Summit Health Center in the Evening on 1/1/12024")
retrieved_docs

[Document(id='8ee5d76d-5740-44af-91a8-e9ae97153687', metadata={'source': 'ER Wait Time Dataset.csv'}, page_content='On 2024-03-19 23:48:51 at Summit Health Center (Region: Urban),\n                urgency level was Low.\n                Total wait time was 100 minutes.\n                Patient outcome was Admitted.\n                Patient satisfaction was 1.'),
 Document(id='0e48a08f-bc3a-49dd-873c-6f8a999e31b9', metadata={'source': 'ER Wait Time Dataset.csv'}, page_content='On 2024-09-17 09:42:50 at Summit Health Center (Region: Urban),\n                urgency level was Low.\n                Total wait time was 120 minutes.\n                Patient outcome was Admitted.\n                Patient satisfaction was 1.'),
 Document(id='c641fc04-97a6-4726-ab4e-a8d683600415', metadata={'source': 'ER Wait Time Dataset.csv'}, page_content='On 2024-09-08 15:41:23 at Summit Health Center (Region: Urban),\n                urgency level was Low.\n                Total wait time was 120 minutes.\

### creating safe Pandas functions

In [19]:
# gets average satisfaction based on any mixed criteria
def average_satisfaction(df, hospital=None, month=None, year=None):
    if hospital:
        filtered = df[df["Hospital Name"] == hospital]

    if month:
        filtered = df[df["Visit Date"].dt.month == month]

    if year:
        filtered = df[df["Visit Date"].dt.year == year]

    if len(filtered) == 0:
        return None

    return round(filtered["Patient Satisfaction"].mean(), 2)

In [20]:
# gets average wait time based on any mixed criteria
def average_wait_time(df, hospital=None, month=None, year=None):
    if hospital:
        filtered = df[df["Hospital Name"] == hospital]

    if month:
        filtered = df[df["Visit Date"].dt.month == month]

    if year:
        filtered = df[df["Visit Date"].dt.year == year]

    if len(filtered) == 0:
        return None

    return round(filtered["Total Wait Time (min)"].mean(), 2)

In [21]:
# function 3!
# function 4!
# function 5!

### initilaizing <b>chat model and prompts</b>

In [22]:
chatModel = ChatOpenAI(model="gpt-4o")

In [23]:
# initilize system prompt
system_prompt = (
    "You are an analytics assistant for question-answering tasks."
    "Use the following pieces of retrieved context to answer"
    "the question. If you don't know the answer, say that you"
    "don't know. User three sentences maximum and keep the"
    "answer concise"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")

    ]
)

### parameters extraction for analytical questions

In [24]:
# extract parameters from the question for analytical questions
# this is done after initilizing of chat model as we are using it to extract
def extract_parameters(question):
    extraction_prompt = f"""
    Extract the following from the question:
    - hospital
    - month (as number)
    - year (as number)

    Return in the format: hospital=<name>; month=<number>; year=<number>
    Question: "{question}"
    """
    response = chatModel.invoke(extraction_prompt)
    text = response.content.strip()

    params = {}
    for part in text.split(";"):
        if "=" in part:
            key, value = part.split("=")
            key = key.strip()
            value = value.strip()
            if value.lower() in ["none", "null"]:
                value = None
            elif key in ["month", "year"]:
                value = int(value)
            params[key] = value
    return params

### whole RAG pipeline

In [25]:
# merge all relevant documents into single return prompt for LLM
question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
# full RAG pipeline (Retrival + Generation)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

### main chatbot code

In [26]:
def chatbot(question):
    # ask chatbot to classifiy intention of question
    intent_prompt = f"""
    Classify the user question as either:
    - 'analytical' (requires math, average, count, calculation)
    - 'descriptive' (explanation or general info)

    Question: {question}
    Only return one word: 'analytical' or 'descriptive'.
    """
    intent_response = chatModel.invoke(intent_prompt)
    intent = intent_response.content.strip().lower()  # FIXED

    # if analytical, use safe Pandas functions
    if intent == "analytical":

        params = extract_parameters(question)
        hospital = params.get("hospital")
        month = params.get("month")
        year = params.get("year")

        result = average_satisfaction(df, hospital, month, year)

        if result is None:
            return "No data found for that query."

        formatting_prompt = f"The calculated average satisfaction score is {result}. Answer clearly in one sentence."
        formatted = chatModel.invoke(formatting_prompt)
        return formatted.content

    # if descriptive, use RAG
    else:
        response = rag_chain.invoke({"input": question})
        return response["answer"]

#### Testing of Chatbot

In [ ]:
# question 1
question = "What is the average customer satisfaction score for Summit Health Center in Jan 2024?"
answer = chatbot(question)
print(answer)

The calculated average satisfaction score of 2.77 indicates a generally low to moderate level of satisfaction.


In [33]:
# checking question 1
print(average_satisfaction(df,"Summit Health Center", 1, 2024))

2.77
